In [210]:
import requests as req
import pandas as pd
import tweepy
import json
import time
import matplotlib.pyplot as plt
import os
import sqlalchemy

# 1- data gathering:

## 1.1 - archive_df data
- the following data is downlaoded manually

In [211]:
archive_df=pd.read_csv('data/twitter-archive-enhanced.csv')
tweet_ids =list(archive_df['tweet_id'])

In [212]:
archive_df.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


## 1.2 - image predictions data 
- downlaod programmaticaly

In [213]:
downlaod image predictions data
folder_name = 'data'
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

url='https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'

In [214]:
res=req.get(url)
with open(os.path.join(folder_name,'image-predictions.tsv'),mode='wb') as file:
    file.write(res.content)

In [215]:
predictions_df=pd.read_csv('data/image-predictions.tsv',delimiter='\t')

In [216]:
predictions_df.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


## 1.3 - json data from twitter api

In [217]:
#get json data from twitter api
consumer_key = ''
consumer_secret = ''
access_secret = ''
access_token = ''

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth, wait_on_rate_limit = True, wait_on_rate_limit_notify = True)

In [218]:
with open('tweet_json.txt', 'w') as out_file:
    t = time.process_time()
    for tweet_id in tweet_ids:
        try:
            current_tweet = api.get_status(tweet_id,tweet_mode='extended')
            json.dump(current_tweet._json, out_file)
            out_file.write('\n')
        except Exception as e:
            print(tweet_id,e)

In [219]:
#read tweet_json and add to list

tweets_list = []

tweet_json = open('data/tweet_json.txt', "r")

for line in tweet_json:
    try:
        tweet = json.loads(line)
        tweets_list.append(tweet)
    except:
        continue
        
tweet_json.close()

In [220]:
list(archive_df)

['tweet_id',
 'in_reply_to_status_id',
 'in_reply_to_user_id',
 'timestamp',
 'source',
 'text',
 'retweeted_status_id',
 'retweeted_status_user_id',
 'retweeted_status_timestamp',
 'expanded_urls',
 'rating_numerator',
 'rating_denominator',
 'name',
 'doggo',
 'floofer',
 'pupper',
 'puppo']

In [221]:
tweets_df = pd.DataFrame(tweets_list)
tweets_df=tweets_df[['id','retweet_count','favorite_count','created_at']]
tweets_df.to_csv('data/tweet_json.csv')

In [222]:
tweets_df.head()

,id,retweet_count,favorite_count,created_at
0,892420643555336193,8351,38166,Tue Aug 01 16:23:56 +0000 2017
1,892177421306343426,6171,32731,Tue Aug 01 00:17:27 +0000 2017
2,891815181378084864,4082,24638,Mon Jul 31 00:18:03 +0000 2017
3,891689557279858688,8496,41496,Sun Jul 30 15:58:51 +0000 2017
4,891327558926688256,9186,39674,Sat Jul 29 16:00:24 +0000 2017


### now the required three data frames are loaded.


# 2- data assessing :

## 2.1- archive_df data

In [223]:
archive_df.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


In [224]:
archive_df.tail()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
2351,666049248165822465,NaN,NaN,2015-11-16 00:24:50 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we have a 1949 1st generation vulpix. Enj...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666049248...,5,10,None,None,None,None,None
2352,666044226329800704,NaN,NaN,2015-11-16 00:04:52 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is a purebred Piers Morgan. Loves to Netf...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666044226...,6,10,a,None,None,None,None
2353,666033412701032449,NaN,NaN,2015-11-15 23:21:54 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here is a very happy pup. Big fan of well-main...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666033412...,9,10,a,None,None,None,None
2354,666029285002620928,NaN,NaN,2015-11-15 23:05:30 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is a western brown Mitsubishi terrier. Up...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666029285...,7,10,a,None,None,None,None
2355,666020888022790149,NaN,NaN,2015-11-15 22:32:08 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we have a Japanese Irish Setter. Lost eye...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666020888...,8,10,None,None,None,None,None


In [225]:
archive_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

In [226]:
archive_df.source.value_counts

<bound method IndexOpsMixin.value_counts of 0       <a href="http://twitter.com/download/iphone" r...
1       <a href="http://twitter.com/download/iphone" r...
2       <a href="http://twitter.com/download/iphone" r...
3       <a href="http://twitter.com/download/iphone" r...
4       <a href="http://twitter.com/download/iphone" r...
5       <a href="http://twitter.com/download/iphone" r...
6       <a href="http://twitter.com/download/iphone" r...
7       <a href="http://twitter.com/download/iphone" r...
8       <a href="http://twitter.com/download/iphone" r...
9       <a href="http://twitter.com/download/iphone" r...
10      <a href="http://twitter.com/download/iphone" r...
11      <a href="http://twitter.com/download/iphone" r...
12      <a href="http://twitter.com/download/iphone" r...
13      <a href="http://twitter.com/download/iphone" r...
14      <a href="http://twitter.com/download/iphone" r...
15      <a href="http://twitter.com/download/iphone" r...
16      <a href="http://twit

In [227]:
len(archive_df[archive_df.duplicated()])

0

no duplicates!

### 2.1.1- structural problems - Tidiness   issues:
- rating_numerator and rating_denominator can be stored in one column , name it by rating_percent , which generated by the following formula : rating_numerator/rating_denominator.
- doggo, floofer, pupper, and puppo columns can combined in one column ! all of theis are categorical data , we can combine them in one categorical column named it by stage.

### 2.1.2- Quality problems - dirty  issues:
- the following columns have a lot of nulls : in_reply_to_status_id , in_reply_to_user_id , retweeted_status_id , retweeted_status_user_id , retweeted_status_timestamp . At the same time , there is no need for theis columns .
- id's (tweet_id , in_reply_to_status_id , in_reply_to_user_id , retweeted_status_id , retweeted_status_user_id ) type is numeric(int or float) , id should be string better than numeric.
- retweeted_status_timestamp and timestamp are dates , but they stored as objects (string).
- source column storing a very long text with tag containing the source , we can store the source in few words instead (for example : iphone , chrome , android ... etc. )
- doggo, floofer, pupper, and puppo columns have None value rather than null .
- expanded_urls has some missing values.

## 2.2- image predictions data 

In [228]:
predictions_df.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


In [229]:
predictions_df.jpg_url.value_counts

<bound method IndexOpsMixin.value_counts of 0         https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg
1         https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg
2         https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg
3         https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg
4         https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg
5         https://pbs.twimg.com/media/CT5Jof1WUAEuVxN.jpg
6         https://pbs.twimg.com/media/CT5KoJ1WoAAJash.jpg
7         https://pbs.twimg.com/media/CT5N9tpXIAAifs1.jpg
8         https://pbs.twimg.com/media/CT5PY90WoAAQGLo.jpg
9         https://pbs.twimg.com/media/CT5Qw94XAAA_2dP.jpg
10        https://pbs.twimg.com/media/CT5Vg_wXIAAXfnj.jpg
11        https://pbs.twimg.com/media/CT5cN_3WEAAlOoZ.jpg
12        https://pbs.twimg.com/media/CT5d9DZXAAALcwe.jpg
13        https://pbs.twimg.com/media/CT5m4VGWEAAtKc8.jpg
14        https://pbs.twimg.com/media/CT5w9gUW4AAsBNN.jpg
15        https://pbs.twimg.com/media/CT51-JJUEAA6hV8.jpg
16        https://pbs.twimg.

In [230]:
predictions_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [231]:
predictions_df[predictions_df.duplicated()]

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog


- no duplicates 

### 2.2.1- structural problems - Tidiness   issues:
- columns names are not readable (shrtcuts are not known!). 

### 2.2.2- Quality problems - dirty  issues:
- for image_url column , the first part ( https://pbs.twimg.com/) is common for all rows , so we can save the link after this part to reduce the space of saved data.
- tweet_id type is int not string.


## 2.3-  json data from twitter api

In [232]:
tweets_df.head()

,id,retweet_count,favorite_count,created_at
0,892420643555336193,8351,38166,Tue Aug 01 16:23:56 +0000 2017
1,892177421306343426,6171,32731,Tue Aug 01 00:17:27 +0000 2017
2,891815181378084864,4082,24638,Mon Jul 31 00:18:03 +0000 2017
3,891689557279858688,8496,41496,Sun Jul 30 15:58:51 +0000 2017
4,891327558926688256,9186,39674,Sat Jul 29 16:00:24 +0000 2017


In [233]:
tweets_df.tail()

,id,retweet_count,favorite_count,created_at
1672,681523177663676416,6245,15058,Mon Dec 28 17:12:42 +0000 2015
1673,681340665377193984,299,1727,Mon Dec 28 05:07:27 +0000 2015
1674,681339448655802368,4357,9772,Mon Dec 28 05:02:37 +0000 2015
1675,681320187870711809,811,2793,Mon Dec 28 03:46:05 +0000 2015
1676,681302363064414209,1962,4145,Mon Dec 28 02:35:15 +0000 2015


In [234]:
tweets_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1677 entries, 0 to 1676
Data columns (total 4 columns):
id                1677 non-null int64
retweet_count     1677 non-null int64
favorite_count    1677 non-null int64
created_at        1677 non-null object
dtypes: int64(3), object(1)
memory usage: 52.5+ KB


In [235]:
tweets_df[tweets_df.duplicated()]

,id,retweet_count,favorite_count,created_at


- no duplicates

### 2.3.1- structural problems - Tidiness   issues:
- created_at is already saved in archive_df table.


### 2.3.1- Quality problems - dirty  issues:

- id type is int not string.

# 3- data cleaning:

In [236]:
clean_predictions=predictions_df.copy()

In [237]:
clean_archive=archive_df.copy()

In [238]:
clean_tweets=tweets_df.copy()

In [239]:
clean_archive.head(3)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None


## 3.1 structural problems - Tidiness   issues:
- 4 problems are cleaned .

### - archive_df data

### 3.1.1- define:
- rating_numerator and rating_denominator can be stored in one column , name it by rating_percent , which generated by the following formula : rating_numerator/rating_denominator.

### - code:

In [240]:
clean_archive['rating_ratio']=clean_archive['rating_numerator']/clean_archive['rating_denominator']

In [241]:
clean_archive.describe()['rating_ratio']

count    2356.000000
mean             inf
std              NaN
min         0.000000
25%         1.000000
50%         1.100000
75%         1.200000
max              inf
Name: rating_ratio, dtype: float64

- their is infinite values of the previous step , this is because division by zero , so I will add a very small value to the denominator.

In [242]:
clean_archive['rating_ratio']=clean_archive['rating_numerator']/(clean_archive['rating_denominator']+0.1)

In [243]:
clean_archive.describe()['rating_ratio']

count    2356.000000
mean        5.284164
std       197.796934
min         0.000000
25%         0.990099
50%         1.089109
75%         1.188119
max      9600.000000
Name: rating_ratio, dtype: float64

- mathmatically , the numerator should be less than or equal to denominator to calculate the ratio , but in the given data set this rule is'nt applied , see the following article : https://knowyourmeme.com/memes/theyre-good-dogs-brent 

- now , I will drop rating_numerator and rating_denominator columns.

In [244]:
clean_archive=clean_archive.drop(['rating_numerator', 'rating_denominator'],axis=1)

In [245]:
list(clean_archive)

['tweet_id',
 'in_reply_to_status_id',
 'in_reply_to_user_id',
 'timestamp',
 'source',
 'text',
 'retweeted_status_id',
 'retweeted_status_user_id',
 'retweeted_status_timestamp',
 'expanded_urls',
 'name',
 'doggo',
 'floofer',
 'pupper',
 'puppo',
 'rating_ratio']

### 3.1.2- define :
- doggo, floofer, pupper, and puppo columns can combined in one column ! all of theis are categorical data , we can combine them in one categorical column named it by stage. 

### - code 

In [246]:
for index, row in clean_archive.iterrows():
    floofer=0
    doggo=0
    puppo=0
    pupper=0
    if clean_archive.at[index, 'doggo']=='doggo':
        clean_archive.at[index, 'stage']='doggo'
        doggo=doggo+1
    if clean_archive.at[index, 'floofer']=='floofer':
        clean_archive.at[index, 'stage']='floofer'
        floofer=floofer+1
    if clean_archive.at[index, 'pupper']=='pupper':
        clean_archive.at[index, 'stage']='pupper'
        pupper=pupper+1
    if clean_archive.at[index, 'puppo']=='puppo':
        clean_archive.at[index, 'stage']='puppo'
        puppo=puppo+1
        #now for multiple stages in the same row: 
    if (floofer+doggo+puppo+pupper)>1:
        clean_archive.at[index, 'stage']='multiple'

In [247]:
# to category
clean_archive.stage=clean_archive.stage.astype('category')

In [248]:
clean_archive.stage.value_counts()

pupper      245
doggo        83
puppo        29
multiple     14
floofer       9
Name: stage, dtype: int64

In [249]:
sum(clean_archive.stage.isnull())

1976

- the edit is already done , now I will drop the following columns : floofer, doggo, puppo , pupper .
- note that stage column remain has nulls , this is because data entry problems.

In [250]:
clean_archive=clean_archive.drop(['floofer', 'doggo', 'puppo' , 'pupper'],axis=1)

In [251]:
list(clean_archive)

['tweet_id',
 'in_reply_to_status_id',
 'in_reply_to_user_id',
 'timestamp',
 'source',
 'text',
 'retweeted_status_id',
 'retweeted_status_user_id',
 'retweeted_status_timestamp',
 'expanded_urls',
 'name',
 'rating_ratio',
 'stage']

### - image predictions data

### 3.1.3- define :
- columns names are not readable (shortcuts are not known!).

In [252]:
clean_predictions.head(1)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True


### - code

In [253]:
rename_dict={
    'p1':'prediction1',
    'p1_conf':'prediction1_conf',
    'p1_dog':'prediction1_dog',
    'p2':'prediction2',
    'p2_conf':'prediction2_conf',
    'p2_dog':'prediction2_dog',
    'p3':'prediction3',
    'p3_conf':'prediction3_conf',
    'p3_dog':'prediction3_dog'
}
clean_predictions=clean_predictions.rename(index=str, columns=rename_dict)

### -test

In [254]:
clean_predictions.head()

,tweet_id,jpg_url,img_num,prediction1,prediction1_conf,prediction1_dog,prediction2,prediction2_conf,prediction2_dog,prediction3,prediction3_conf,prediction3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


In [255]:
clean_predictions.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id            2075 non-null int64
jpg_url             2075 non-null object
img_num             2075 non-null int64
prediction1         2075 non-null object
prediction1_conf    2075 non-null float64
prediction1_dog     2075 non-null bool
prediction2         2075 non-null object
prediction2_conf    2075 non-null float64
prediction2_dog     2075 non-null bool
prediction3         2075 non-null object
prediction3_conf    2075 non-null float64
prediction3_dog     2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 168.2+ KB


### - json data from twitter api

### 3.1.4 - define
- created_at is already saved in archive_df table.

### - code

In [256]:
clean_tweets=clean_tweets.drop('created_at',axis=1)

### - test

In [257]:
clean_tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1677 entries, 0 to 1676
Data columns (total 3 columns):
id                1677 non-null int64
retweet_count     1677 non-null int64
favorite_count    1677 non-null int64
dtypes: int64(3)
memory usage: 39.4 KB


## 3.2 - Quality problems - dirty issues:
- 8 problems are cleaned as the following

### - archive_df data

### 3.2.1- define
- the following columns have a lot of nulls : in_reply_to_status_id , in_reply_to_user_id , retweeted_status_id , retweeted_status_user_id , retweeted_status_timestamp . At the same time , there is no need for theis columns .

### - code 

In [258]:
clean_archive.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 13 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
name                          2356 non-null object
rating_ratio                  2356 non-null float64
stage                         380 non-null category
dtypes: category(1), float64(5), int64(1), object(6)
memory usage: 223.4+ KB


In [259]:
clean_archive=clean_archive.drop([ 'in_reply_to_status_id' , 'in_reply_to_user_id' , 'retweeted_status_id' , 'retweeted_status_user_id' , 'retweeted_status_timestamp'],axis=1)

### - test

In [260]:
clean_archive.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 8 columns):
tweet_id         2356 non-null int64
timestamp        2356 non-null object
source           2356 non-null object
text             2356 non-null object
expanded_urls    2297 non-null object
name             2356 non-null object
rating_ratio     2356 non-null float64
stage            380 non-null category
dtypes: category(1), float64(1), int64(1), object(5)
memory usage: 131.4+ KB


### 3.2.2- define
- tweet_id type is int not string

### - code

In [261]:
clean_archive.tweet_id=clean_archive.tweet_id.astype('str')

### - test

In [262]:
clean_archive.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 8 columns):
tweet_id         2356 non-null object
timestamp        2356 non-null object
source           2356 non-null object
text             2356 non-null object
expanded_urls    2297 non-null object
name             2356 non-null object
rating_ratio     2356 non-null float64
stage            380 non-null category
dtypes: category(1), float64(1), object(6)
memory usage: 131.4+ KB


In [263]:
clean_archive.tweet_id.head()

0    892420643555336193
1    892177421306343426
2    891815181378084864
3    891689557279858688
4    891327558926688256
Name: tweet_id, dtype: object

###  3.2.3- define
- timestamp is date , but  stored as objects (string)

### - code

In [264]:
clean_archive.timestamp = pd.to_datetime(clean_archive.timestamp)

### - test

In [265]:
clean_archive.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 8 columns):
tweet_id         2356 non-null object
timestamp        2356 non-null datetime64[ns]
source           2356 non-null object
text             2356 non-null object
expanded_urls    2297 non-null object
name             2356 non-null object
rating_ratio     2356 non-null float64
stage            380 non-null category
dtypes: category(1), datetime64[ns](1), float64(1), object(5)
memory usage: 131.4+ KB


In [266]:
clean_archive.head()

,tweet_id,timestamp,source,text,expanded_urls,name,rating_ratio,stage
0,892420643555336193,2017-08-01 16:23:56,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,https://twitter.com/dog_rates/status/892420643...,Phineas,1.287129,NaN
1,892177421306343426,2017-08-01 00:17:27,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,https://twitter.com/dog_rates/status/892177421...,Tilly,1.287129,NaN
2,891815181378084864,2017-07-31 00:18:03,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,https://twitter.com/dog_rates/status/891815181...,Archie,1.188119,NaN
3,891689557279858688,2017-07-30 15:58:51,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,https://twitter.com/dog_rates/status/891689557...,Darla,1.287129,NaN
4,891327558926688256,2017-07-29 16:00:24,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,https://twitter.com/dog_rates/status/891327558...,Franklin,1.188119,NaN


###  3.2.4- define
- source column storing a very long text with tag containing the source , we can store the source in less words instead : Iphone , Vine  , Twitter Web Client and TweetDeck .

In [267]:
clean_archive.source.value_counts()

<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>     2221
<a href="http://vine.co" rel="nofollow">Vine - Make a Scene</a>                          91
<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>                       33
<a href="https://about.twitter.com/products/tweetdeck" rel="nofollow">TweetDeck</a>      11
Name: source, dtype: int64

### - code

In [268]:
sources_list=['<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>',
             '<a href="http://vine.co" rel="nofollow">Vine - Make a Scene</a>',
             '<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>',
             '<a href="https://about.twitter.com/products/tweetdeck" rel="nofollow">TweetDeck</a>'
            ]
source_short_dict={
    sources_list[0] :'iphone',
    sources_list[1] : 'Vine' ,
    sources_list[2] : 'Twitter Web Client',
    sources_list[3] : 'TweetDeck'
}

In [269]:
source_short_dict.keys()

dict_keys(['<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', '<a href="http://vine.co" rel="nofollow">Vine - Make a Scene</a>', '<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>', '<a href="https://about.twitter.com/products/tweetdeck" rel="nofollow">TweetDeck</a>'])

In [270]:
for index, row in clean_archive.iterrows():
    if clean_archive.at[index, 'source']==sources_list[0]:
        clean_archive.at[index, 'source']=source_short_dict[sources_list[0]]
        
    elif clean_archive.at[index, 'source']==sources_list[1]:
        clean_archive.at[index, 'source']=source_short_dict[sources_list[1]]
        
    elif clean_archive.at[index, 'source']==sources_list[2]:
        clean_archive.at[index, 'source']=source_short_dict[sources_list[2]]
    else:
        clean_archive.at[index, 'source']=source_short_dict[sources_list[3]]  

### - test

In [271]:
clean_archive.source.value_counts()

iphone                2221
Vine                    91
Twitter Web Client      33
TweetDeck               11
Name: source, dtype: int64

In [272]:
clean_archive.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 8 columns):
tweet_id         2356 non-null object
timestamp        2356 non-null datetime64[ns]
source           2356 non-null object
text             2356 non-null object
expanded_urls    2297 non-null object
name             2356 non-null object
rating_ratio     2356 non-null float64
stage            380 non-null category
dtypes: category(1), datetime64[ns](1), float64(1), object(5)
memory usage: 131.4+ KB


### - image predictions data 

###  3.2.5- define
- for jpg_url column , the first part ( https://pbs.twimg.com/) is common for all rows , so we can save the link after this part to reduce the space of saved data.

In [273]:
clean_predictions[clean_predictions.jpg_url.str[0:len('https://pbs.twimg.com')]!='https://pbs.twimg.com']

,tweet_id,jpg_url,img_num,prediction1,prediction1_conf,prediction1_dog,prediction2,prediction2_conf,prediction2_dog,prediction3,prediction3_conf,prediction3_dog


### - code

In [274]:
clean_predictions.jpg_url=clean_predictions.jpg_url.str[len('https://pbs.twimg.com'):]

### - test

In [275]:
clean_predictions.jpg_url.head()

0    /media/CT4udn0WwAA0aMy.jpg
1    /media/CT42GRgUYAA5iDo.jpg
2    /media/CT4521TWwAEvMyu.jpg
3    /media/CT5Dr8HUEAA-lEu.jpg
4    /media/CT5IQmsXIAAKY4A.jpg
Name: jpg_url, dtype: object

###  3.2.6- define
- tweet_id type is int not string.

In [276]:
clean_predictions.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id            2075 non-null int64
jpg_url             2075 non-null object
img_num             2075 non-null int64
prediction1         2075 non-null object
prediction1_conf    2075 non-null float64
prediction1_dog     2075 non-null bool
prediction2         2075 non-null object
prediction2_conf    2075 non-null float64
prediction2_dog     2075 non-null bool
prediction3         2075 non-null object
prediction3_conf    2075 non-null float64
prediction3_dog     2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 168.2+ KB


### - code

In [277]:
clean_predictions.tweet_id=clean_predictions.tweet_id.astype('str')

### - test

In [278]:
clean_predictions.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id            2075 non-null object
jpg_url             2075 non-null object
img_num             2075 non-null int64
prediction1         2075 non-null object
prediction1_conf    2075 non-null float64
prediction1_dog     2075 non-null bool
prediction2         2075 non-null object
prediction2_conf    2075 non-null float64
prediction2_dog     2075 non-null bool
prediction3         2075 non-null object
prediction3_conf    2075 non-null float64
prediction3_dog     2075 non-null bool
dtypes: bool(3), float64(3), int64(1), object(5)
memory usage: 168.2+ KB


### 3.2.7- define
- predictions contains under score ( _ ) , I will remove them

In [279]:
clean_predictions.head()

,tweet_id,jpg_url,img_num,prediction1,prediction1_conf,prediction1_dog,prediction2,prediction2_conf,prediction2_dog,prediction3,prediction3_conf,prediction3_dog
0,666020888022790149,/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


### - code

In [280]:
clean_predictions.prediction1=clean_predictions.prediction1.str.replace('_',' ')
clean_predictions.prediction2=clean_predictions.prediction2.str.replace('_',' ')
clean_predictions.prediction3=clean_predictions.prediction3.str.replace('_',' ')

### -test

In [281]:
clean_predictions.head()

,tweet_id,jpg_url,img_num,prediction1,prediction1_conf,prediction1_dog,prediction2,prediction2_conf,prediction2_dog,prediction3,prediction3_conf,prediction3_dog
0,666020888022790149,/media/CT4udn0WwAA0aMy.jpg,1,Welsh springer spaniel,0.465074,True,collie,0.156665,True,Shetland sheepdog,0.061428,True
1,666029285002620928,/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature pinscher,0.074192,True,Rhodesian ridgeback,0.072010,True
2,666033412701032449,/media/CT4521TWwAEvMyu.jpg,1,German shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian ridgeback,0.408143,True,redbone,0.360687,True,miniature pinscher,0.222752,True
4,666049248165822465,/media/CT5IQmsXIAAKY4A.jpg,1,miniature pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


### -  json data from twitter api

###  3.2.8- define
- id type is int not string.

### - code

In [282]:
clean_tweets.id=clean_tweets.id.astype('str')

### - test

In [283]:
clean_tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1677 entries, 0 to 1676
Data columns (total 3 columns):
id                1677 non-null object
retweet_count     1677 non-null int64
favorite_count    1677 non-null int64
dtypes: int64(2), object(1)
memory usage: 39.4+ KB


## 4- data frames are cleaned , now I will merge them togother

In [284]:
final_data_clean = pd.merge(left=clean_archive,
                                 right=clean_tweets, left_on='tweet_id', right_on='id', how='inner')

In [285]:
final_data_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1677 entries, 0 to 1676
Data columns (total 11 columns):
tweet_id          1677 non-null object
timestamp         1677 non-null datetime64[ns]
source            1677 non-null object
text              1677 non-null object
expanded_urls     1629 non-null object
name              1677 non-null object
rating_ratio      1677 non-null float64
stage             317 non-null category
id                1677 non-null object
retweet_count     1677 non-null int64
favorite_count    1677 non-null int64
dtypes: category(1), datetime64[ns](1), float64(1), int64(2), object(6)
memory usage: 146.0+ KB


In [286]:
final_data_clean= pd.merge(left=final_data_clean,
                                 right=clean_predictions, left_on='tweet_id', right_on='tweet_id', how='inner')

In [287]:
final_data_clean.head()

,tweet_id,timestamp,source,text,expanded_urls,name,rating_ratio,stage,id,retweet_count,...,img_num,prediction1,prediction1_conf,prediction1_dog,prediction2,prediction2_conf,prediction2_dog,prediction3,prediction3_conf,prediction3_dog
0,892420643555336193,2017-08-01 16:23:56,iphone,This is Phineas. He's a mystical boy. Only eve...,https://twitter.com/dog_rates/status/892420643...,Phineas,1.287129,NaN,892420643555336193,8351,...,1,orange,0.097049,False,bagel,0.085851,False,banana,0.076110,False
1,892177421306343426,2017-08-01 00:17:27,iphone,This is Tilly. She's just checking pup on you....,https://twitter.com/dog_rates/status/892177421...,Tilly,1.287129,NaN,892177421306343426,6171,...,1,Chihuahua,0.323581,True,Pekinese,0.090647,True,papillon,0.068957,True
2,891815181378084864,2017-07-31 00:18:03,iphone,This is Archie. He is a rare Norwegian Pouncin...,https://twitter.com/dog_rates/status/891815181...,Archie,1.188119,NaN,891815181378084864,4082,...,1,Chihuahua,0.716012,True,malamute,0.078253,True,kelpie,0.031379,True
3,891689557279858688,2017-07-30 15:58:51,iphone,This is Darla. She commenced a snooze mid meal...,https://twitter.com/dog_rates/status/891689557...,Darla,1.287129,NaN,891689557279858688,8496,...,1,paper towel,0.170278,False,Labrador retriever,0.168086,True,spatula,0.040836,False
4,891327558926688256,2017-07-29 16:00:24,iphone,This is Franklin. He would like you to stop ca...,https://twitter.com/dog_rates/status/891327558...,Franklin,1.188119,NaN,891327558926688256,9186,...,2,basset,0.555712,True,English springer,0.225770,True,German short-haired pointer,0.175219,True


In [288]:
final_data_clean=final_data_clean.drop(['id'],axis=1)

In [289]:
final_data_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1426 entries, 0 to 1425
Data columns (total 21 columns):
tweet_id            1426 non-null object
timestamp           1426 non-null datetime64[ns]
source              1426 non-null object
text                1426 non-null object
expanded_urls       1426 non-null object
name                1426 non-null object
rating_ratio        1426 non-null float64
stage               263 non-null category
retweet_count       1426 non-null int64
favorite_count      1426 non-null int64
jpg_url             1426 non-null object
img_num             1426 non-null int64
prediction1         1426 non-null object
prediction1_conf    1426 non-null float64
prediction1_dog     1426 non-null bool
prediction2         1426 non-null object
prediction2_conf    1426 non-null float64
prediction2_dog     1426 non-null bool
prediction3         1426 non-null object
prediction3_conf    1426 non-null float64
prediction3_dog     1426 non-null bool
dtypes: bool(3), category(1)

## 5- export final data 

In [290]:
final_data_clean.to_csv('data/twitter_archive_master.csv',index=False)

## 6- visualizations 

In [291]:
final_data_clean.describe()

,rating_ratio,retweet_count,favorite_count,img_num,prediction1_conf,prediction2_conf,prediction3_conf
count,1426.000000,1426.000000,1426.000000,1426.000000,1426.000000,1426.000000,1.426000e+03
mean,1.236009,3624.610799,11158.836606,1.271388,0.607958,0.135173,5.937607e-02
std,4.634117,5431.670424,13938.056784,0.640160,0.266884,0.101126,5.054457e-02
min,0.000000,23.000000,0.000000,1.000000,0.059033,0.000010,5.595040e-07
25%,0.990099,1077.250000,3193.750000,1.000000,0.377601,0.052717,1.564975e-02
50%,1.099078,2101.000000,6636.500000,1.000000,0.610335,0.119365,4.813825e-02
75%,1.188119,4062.500000,14112.750000,1.000000,0.856331,0.196461,9.166433e-02
max,175.841584,83940.000000,164748.000000,4.000000,0.999984,0.488014,2.734190e-01


In [292]:
final_data_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1426 entries, 0 to 1425
Data columns (total 21 columns):
tweet_id            1426 non-null object
timestamp           1426 non-null datetime64[ns]
source              1426 non-null object
text                1426 non-null object
expanded_urls       1426 non-null object
name                1426 non-null object
rating_ratio        1426 non-null float64
stage               263 non-null category
retweet_count       1426 non-null int64
favorite_count      1426 non-null int64
jpg_url             1426 non-null object
img_num             1426 non-null int64
prediction1         1426 non-null object
prediction1_conf    1426 non-null float64
prediction1_dog     1426 non-null bool
prediction2         1426 non-null object
prediction2_conf    1426 non-null float64
prediction2_dog     1426 non-null bool
prediction3         1426 non-null object
prediction3_conf    1426 non-null float64
prediction3_dog     1426 non-null bool
dtypes: bool(3), category(1)

- I created the visualization using tableau , see the following link : 
<a href='https://public.tableau.com/profile/rami.salman#!/vizhome/weRateDogs/WeRateDogsStory?publish=yes'>WeRateDog tableau visualzation<a/>


## 7- insights

- 1 : number of records decreases by forwarding days.
- 2 : average of favourite number increases by forwarding days.
- 3 : average of rating ratio increases in very slow rate by forwarding days.
- 4 : the highest number of tweets is on monday followed by tuesday , on the other hand , the lowest number of tweets is on Saturday.
- 5 : the highest average rating ratio is on Monday.
- 6 : the highest avg. favourite count is for puppo dogs.
- 7 : the highest number of records is for pupper dogs.
- 8 : the highest average rating ratio is for floofer dogs.
- 9 : most of tweets are from iphone.
- 10 : the tweets uploaded from tweetDeck have the highest rating ratio of 26.1 .